<a href="https://colab.research.google.com/github/anastasiia784/Data-Analyst-/blob/main/Portfolio_Project_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Portfolio Project 2

**Goal:**
Build a scalable Python-based tool to calculate statistical significance for multiple A/B test conversion metrics without hardcoding the number of metrics.
The results should be ready for further visualization (Tableau / BI tools).

### Import Required Libraries
Import libraries for data processing and statistical testing.

In [2]:
# Required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.stats.proportion import proportions_ztest


from google.colab import drive
drive.mount("/content/drive")

%cd /content/drive/MyDrive/Mate_homework




Mounted at /content/drive
/content/drive/MyDrive/Mate_homework


### Load the Dataset
Load the dataset

In [3]:
# Load dataset
df = pd.read_csv("Portfolio_project_2.csv")

# Preview data structure
df.head()



,date,country,device,continent,channel,test,test_group,event_name,value
0,2020-11-01,United States,desktop,Americas,Organic Search,2,2,sessions with orders,14
1,2020-11-01,United States,desktop,Americas,Direct,2,2,sessions with orders,8
2,2020-11-01,India,desktop,Asia,Organic Search,2,2,sessions with orders,2
3,2020-11-01,Japan,mobile,Asia,Direct,2,1,sessions with orders,1
4,2020-11-01,United Arab Emirates,mobile,Asia,Organic Search,2,1,sessions with orders,1


### Inspect Available Event Names
Confirm all available event names in the dataset.

In [4]:
df['event_name'].unique()


array(['sessions with orders', 'new account', 'session', 'scroll',
       'select_promotion', 'session_start', 'user_engagement',
       'first_visit', 'page_view', 'view_item', 'view_promotion',
       'view_search_results', 'add_payment_info', 'add_shipping_info',
       'begin_checkout', 'select_item', 'add_to_cart', 'click',
       'view_item_list'], dtype=object)

### Define Conversion Metrics
Statically define required metrics using actual event names.


In [5]:
metrics = {
    'add_payment': 'add_payment_info',
    'add_shipping': 'add_shipping_info',
    'begin_checkout': 'begin_checkout',
    'new_accounts': 'new account'
}

SESSION_EVENT = 'session'
VALUE_COL = 'value'


### Aggregate Event Counts
Aggregate event counts for statistical calculations

In [6]:
pivot = (
    df
    .pivot_table(
        index=['test', 'test_group', 'event_name'],
        values=VALUE_COL,
        aggfunc='sum'
    )
    .reset_index()
)

pivot.head()


,test,test_group,event_name,value
0,1,1,add_payment_info,1988
1,1,1,add_shipping_info,3034
2,1,1,add_to_cart,1395
3,1,1,begin_checkout,3784
4,1,1,click,368


### Create Z-test Function for Proportions

Create a reusable function for A/B test significance testing.

In [7]:
def run_z_test(success_c, success_t, total_c, total_t):
    successes = np.array([success_c, success_t])
    totals = np.array([total_c, total_t])
    z_stat, p_value = proportions_ztest(successes, totals)
    return z_stat, p_value


### Calculate Statistical Significance for All Metrics

Loop through all metrics dynamically and calculate:

conversion rates

absolute and relative differences

statistical significance

In [8]:
results = []

for test_number in pivot['test'].unique():

    for metric_name, event_name in metrics.items():

        # Control group = 1, Test group = 2
        numerator_control = pivot.query(
            "test == @test_number and test_group == 1 and event_name == @event_name"
        )[VALUE_COL].sum()

        numerator_test = pivot.query(
            "test == @test_number and test_group == 2 and event_name == @event_name"
        )[VALUE_COL].sum()

        denominator_control = pivot.query(
            "test == @test_number and test_group == 1 and event_name == @SESSION_EVENT"
        )[VALUE_COL].sum()

        denominator_test = pivot.query(
            "test == @test_number and test_group == 2 and event_name == @SESSION_EVENT"
        )[VALUE_COL].sum()

        # Conversion rates
        cr_control = numerator_control / denominator_control
        cr_test = numerator_test / denominator_test

        # Metric change (%)
        metric_change = (cr_test - cr_control) / cr_control * 100

        # Statistical test
        z_stat, p_value = run_z_test(
            numerator_control,
            numerator_test,
            denominator_control,
            denominator_test
        )

        results.append({
            'test': test_number,
            'metrics': f'{metric_name}/session',
            'numerator_event': event_name,
            'denominator_event': SESSION_EVENT,
            'numerator_control': numerator_control,
            'denominator_control': denominator_control,
            'conversion_rate_control': cr_control,
            'numerator_test': numerator_test,
            'denominator_test': denominator_test,
            'conversion_rate_test': cr_test,
            'metric_change_%': metric_change,
            'z_stat': z_stat,
            'p_value': p_value,
            'significant': p_value < 0.05
        })


### Create Final Results Table

In [9]:
final_results_df = pd.DataFrame(results)
final_results_df


,test,metrics,numerator_event,denominator_event,numerator_control,denominator_control,conversion_rate_control,numerator_test,denominator_test,conversion_rate_test,metric_change_%,z_stat,p_value,significant
0,1,add_payment/session,add_payment_info,session,1988,45362,0.043825,2229,45193,0.049322,12.542021,-3.924884,0.000087,True
1,1,add_shipping/session,add_shipping_info,session,3034,45362,0.066884,3221,45193,0.071272,6.560481,-2.603571,0.009226,True
2,1,begin_checkout/session,begin_checkout,session,3784,45362,0.083418,4021,45193,0.088974,6.660587,-2.978783,0.002894,True
3,1,new_accounts/session,new account,session,3823,45362,0.084278,3681,45193,0.081451,-3.354299,1.542883,0.122859,False
4,2,add_payment/session,add_payment_info,session,2344,50637,0.046290,2409,50244,0.047946,3.576911,-1.240994,0.214608,False
5,2,add_shipping/session,add_shipping_info,session,3480,50637,0.068724,3510,50244,0.069859,1.650995,-0.709557,0.477979,False
6,2,begin_checkout/session,begin_checkout,session,4262,50637,0.084168,4313,50244,0.085841,1.988164,-0.952898,0.340642,False
7,2,new_accounts/session,new account,session,4165,50637,0.082252,4184,50244,0.083274,1.241934,-0.588793,0.556000,False
8,3,add_payment/session,add_payment_info,session,3623,70047,0.051722,3697,70439,0.052485,1.474630,-0.643172,0.520112,False
9,3,add_shipping/session,add_shipping_info,session,5298,70047,0.075635,5188,70439,0.073652,-2.621211,1.413727,0.157442,False


### Save Results for BI Visualization

In [ ]:
final_results_df.to_csv('ab_test_final_results.csv', index=False)


### **Tableau Visualization**

 https://public.tableau.com/views/ABTest_17651154233200/ABTest?:language=en-US&publish=yes&:sid=&:redirect=auth&:display_count=n&:origin=viz_share_link

Final results file
https://drive.google.com/file/d/19CK-XlhtD-p3nok-bep4iG9NEuUntRvv/view?usp=sharing